# Import Modules

In [14]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

import utils

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare Data

In [16]:
train_source_X = pd.read_csv("./deep_occupancy_detection/data/2_X_train.csv").values
target_X = pd.read_csv("./deep_occupancy_detection/data/1_X_train.csv").values
train_source_y_task = pd.read_csv("./deep_occupancy_detection/data/2_Y_train.csv").values.reshape(-1)
target_y_task = pd.read_csv("./deep_occupancy_detection/data/1_Y_train.csv").values.reshape(-1)

scaler = preprocessing.StandardScaler()
train_source_X = scaler.fit_transform(train_source_X)
target_X = scaler.fit_transform(target_X)

In [17]:
train_source_X, train_source_y_task = utils.apply_sliding_window(train_source_X, train_source_y_task, filter_len=3)
target_X, target_y_task = utils.apply_sliding_window(target_X, target_y_task, filter_len=3)
_, test_target_X, _, test_target_y_task = train_test_split(target_X, target_y_task, test_size=0.3, shuffle=False)

In [18]:
train_source_X = torch.tensor(train_source_X, dtype=torch.float32)
train_source_y_task = torch.tensor(train_source_y_task, dtype=torch.float32)
test_target_X = torch.tensor(test_target_X, dtype=torch.float32)
test_target_y_task = torch.tensor(test_target_y_task, dtype=torch.float32)

train_source_X = train_source_X.to(device)
train_source_y_task = train_source_y_task.to(device)
test_target_X = test_target_X.to(device)
test_target_y_task = test_target_y_task.to(device)

source_ds = TensorDataset(train_source_X, train_source_y_task)
source_loader = DataLoader(source_ds, batch_size=34, shuffle=True)